# Yelp Api
import statements

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import json
import csv

Find website function

In [139]:
def find_url(yelp_url):

    # Send a GET request to the website
    response = requests.get(yelp_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML response
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Find all elements with the specified class
        elements = soup.find_all(class_='css-1idmmu3')
    
        # Extract the link from the first element
        if elements:
            for element in elements:
                # print(element.text)
                site = re.search(r'\b[a-zA-Z0-9-]+\.[a-zA-Z]{3}\b', element.text)
                if site:
                    return(site.group())
                else:
                    pass
        else:
            print('No elements found with the specified class')
    else:
        print(f'Error: {response.status_code}')

standard stuff

In [140]:
# Set the API key
API_KEY = 'QYyH-sUOrRJM68JIaRBiP47EFzwQz9brjUuQnBsbrhKtlzXRW58WAcWpK6qZKTZF-CPu5iRKLXMXN_zLY0H7otLn-d_SBJM8KiE66fvXJpHr4sHgmHyDw3aQ5nrdZXYx'

# Define the search parameters
term = 'General contractors'
location = 'Madison, WI'

In [141]:
# Define the number of results per page
limit = 50
results = []
# Define the total number of results to retrieve
total_results_to_retrieve = 1000

# Calculate the number of pages needed to retrieve all the results
num_pages = total_results_to_retrieve // limit
if total_results_to_retrieve % limit != 0:
    num_pages += 1

# Make a GET request for each page of results
for page in range(num_pages):
    # Calculate the offset for the current page
    offset = page * limit

    # Define the API endpoint for the current page
    url = f'https://api.yelp.com/v3/businesses/search?term={term}&location={location}&limit={limit}&offset={offset}'

    # Define the headers with the API key
    headers = {
        'Authorization': f'Bearer {API_KEY}'
    }

    # Make a GET request to the API endpoint
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract the businesses
        businesses = data.get('businesses', [])

        # Print the business names and addresses
        for business in businesses:
            name = business.get('name')
            address = ', '.join(business.get('location', {}).get('display_address', []))
            print(f'Name: {name}')
            print(f'Address: {address}')
            print()
            results.append(business)
    else:
        print(f'Error: {response.status_code}')

Name: DC Brothers Repair and Remodel
Address: Madison, WI 53703

Name: Non-Typical Remodeling
Address: Madison, WI 53704

Name: Hannacks Home Services
Address: Fitchburg, WI 53713

Name: BlackSun Builders
Address: Madison, WI 53704

Name: Washa Remodeling and Design
Address: 2605 S Stoughton Rd, Ste 300, Madison, WI 53716

Name: Westring Construction
Address: 4617 Dovetail Dr, Ste 8, Madison, WI 53704

Name: Frey Construction & Home Improvement
Address: 530 Park Ave, Prairie du Sac, WI 53578

Name: 3rd Gen Painting and Remodeling Madison WI
Address: 2201 Carling Dr, Ste 101, Madison, WI 53711

Name: Home MD Property Maintenance & Handyman Services
Address: Evansville, WI 53536

Name: Degnan Design-Build-Remodel
Address: 128 Commerce St, DeForest, WI 53532

Name: Time 2 Remodel
Address: 1218 Gilson St, Madison, WI 53715

Name: JL General Contractors
Address: 1017 Jonathon Dr, Madison, WI 53713

Name: Kitchen Solvers of Madison
Address: 5900 Monona Dr, Ste 405, Monona, WI 53716

Name: Ca

In [142]:
# Define the file path
file_path = 'data/yelp_results.json'

# Open the file in write mode
with open(file_path, 'w') as file:
    # Use the json.dump() function to write the data to the file
    json.dump(data, file)

print(f'Data saved to {file_path}')

Data saved to data/yelp_results.json


In [143]:
results[1]

{'id': 'L7l-NrvbX3i3xmjxh4fW2A',
 'alias': 'non-typical-remodeling-madison',
 'name': 'Non-Typical Remodeling',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/wtLZCKSN0a0LDceclusN1A/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/non-typical-remodeling-madison?adjust_creative=JLLA2lyDuL0kW-S7iYIymA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=JLLA2lyDuL0kW-S7iYIymA',
 'review_count': 2,
 'categories': [{'alias': 'contractors', 'title': 'General Contractors'},
  {'alias': 'flooring', 'title': 'Flooring'},
  {'alias': 'doorsales', 'title': 'Door Sales/Installation'}],
 'rating': 5.0,
 'coordinates': {'latitude': 43.0961456298828, 'longitude': -89.3517379760742},
 'transactions': [],
 'location': {'address1': '',
  'address2': None,
  'address3': '',
  'city': 'Madison',
  'zip_code': '53704',
  'country': 'US',
  'state': 'WI',
  'display_address': ['Madison, WI 53704']},
 'phone': '+16084466532',
 'display_phone': '(608) 446-6532',
 'dista

In [144]:
filtered_results = []
for result in results:
    # print(result)
    location_dict = {}
    try:
        location_dict["name"] = result["name"]
    except KeyError as e:
        print(f'KeyError: {e}')
        pass
    try:
        location_dict["yelp_url"] = result["url"]
    except KeyError as e:
        print(f'KeyError: {e}')
        pass   
    try:
        location_dict["city"] = result["location"]['city']
    except KeyError as e:
        print(f'KeyError: {e}')
        pass
    try:
        location_dict["phone"] = result["phone"]
    except KeyError as e:
        print(f'KeyError: {e}')
        pass
    location_dict["website"] = find_url(result["url"])
    location_dict["email"] = ""

    filtered_results.append(location_dict)
    print(str(len(filtered_results)) + "/" + str(len(results)))

1/422
2/422
3/422
4/422
5/422
6/422
7/422
8/422
9/422
10/422
11/422
12/422
13/422
14/422
15/422
16/422
No elements found with the specified class
17/422
18/422
19/422
20/422
21/422
22/422
23/422
24/422
25/422
26/422
27/422
28/422
29/422
30/422
31/422
32/422
33/422
34/422
35/422
36/422
37/422
38/422
39/422
40/422
41/422
42/422
43/422
44/422
45/422
46/422
47/422
48/422
49/422
50/422
51/422
52/422
53/422
54/422
55/422
56/422
57/422
58/422
59/422
60/422
61/422
62/422
63/422
64/422
65/422
66/422
67/422
68/422
69/422
70/422
71/422
72/422
73/422
No elements found with the specified class
74/422
75/422
76/422
77/422
78/422
79/422
80/422
81/422
82/422
83/422
84/422
85/422
86/422
87/422
88/422
89/422
90/422
91/422
92/422
93/422
94/422
95/422
96/422
97/422
98/422
99/422
100/422
101/422
102/422
103/422
104/422
105/422
106/422
107/422
108/422
109/422
110/422
111/422
112/422
113/422
114/422
115/422
116/422
117/422
118/422
119/422
120/422
121/422
122/422
No elements found with the specified class
123

In [145]:
# Define the file path
file_path = 'data/yelp_results_url_added.json'

# Open the file in write mode
with open(file_path, 'w') as file:
    # Use the json.dump() function to write the data to the file
    json.dump(filtered_results, file)

print(f'Data saved to {file_path}')

Data saved to data/yelp_results_url_added.json


In [146]:
# Key to check for null values
key_to_check = 'website'

# Count the number of null values for the specified key
null_count = sum(1 for d in filtered_results if d.get(key_to_check) is not None)

print(f"Number of '{key_to_check}' found: {null_count}")

Number of 'website' found: 333


In [147]:
# Parse the JSON data into a list of dictionaries
with open("data/yelp_results_url_added.json", 'r') as json_file:
    filtered_results = json.load(json_file)

#### Function to go to found website and see if there is text that matches the standard email address format

In [148]:
def find_email(website_url):
    try:
        # URL of the website to scrape
        url = 'https://' + website_url

        # Send a GET request to the website
        response = requests.get(url)

        # Raise an exception if the response status code is not 200 (OK)
        response.raise_for_status()

        # Parse the HTML content of the website
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all text on the website
        all_text = soup.get_text()

        # Regular expression to match email addresses
        email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{3}\b'

        # Find all email addresses in the text
        email_addresses = re.findall(email_regex, all_text)

        # Check if any email addresses were found
        if email_addresses:
            return email_addresses
        else:
            # print(f"No email addresses found on {url}")
            pass
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [150]:
for company in filtered_results:
    website = company.get("website")
    if website == None:
        print("No Website Found!")
    else:
        emails_found = find_email(website)
        if emails_found != None:
            index = filtered_results.index(company)
            filtered_results[index]["email"] = emails_found
        else:
            emails_found = find_email(website + '/contact/')
            if emails_found != None:
                index = filtered_results.index(company)
                filtered_results[index]["email"] = emails_found
            else: 
                emails_found = find_email(website + "/contact-us/")
                if emails_found != None:
                    index = filtered_results.index(company)
                    filtered_results[index]["email"] = emails_found
                else:
                    print(f"No email found for {website}")

    print("emails searched for" + str(index) + "/" + str(len(filtered_results)))


No Website Found!
emails searched for2/422
No Website Found!
emails searched for2/422
emails searched for2/422
An error occurred: HTTPSConnectionPool(host='blacksunbuilders.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)')))
An error occurred: HTTPSConnectionPool(host='blacksunbuilders.com', port=443): Max retries exceeded with url: /contact/ (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006)')))
An error occurred: HTTPSConnectionPool(host='blacksunbuilders.com', port=443): Max retries exceeded with url: /contact-us/ (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_INTERNAL_ERROR] tlsv1 alert internal error (_ssl.c:1006)')))
No email found for blacksunbuilders.com
emails searched for2/422
No email found for washaremodeling.com
emails searched for2/422
HTTP error occurred: 404 Client Error: Not Found for 

In [151]:
# Define the file path
file_path = 'data/yelp_results_email_added.json'

# Open the file in write mode
with open(file_path, 'w') as file:
    # Use the json.dump() function to write the data to the file
    json.dump(filtered_results, file)

print(f'Data saved to {file_path}')

Data saved to data/yelp_results_email_added.json


In [153]:
key_to_check = 'email'

# Count the number of null values for the specified key
null_count = sum(1 for d in filtered_results if d.get(key_to_check) != "")

print(f"Number of '{key_to_check}' found: {null_count}")

Number of 'email' found: 74


In [12]:
# Parse the JSON data into a list of dictionaries
with open("data/yelp_results_w_email_only.json", 'r') as json_file:
    ready_for_csv = json.load(json_file)

In [13]:
for company in ready_for_csv:
    company.pop('yelp_url')
    if len(company['email']) ==1:
        company['email'] = company['email'][0]
    else:
        pass

In [14]:

# Specify the field names (keys of the dictionaries)
fieldnames = ready_for_csv[0].keys()

# Specify the name of the CSV file
filename = 'final_output.csv'

# Open the CSV file in write mode
with open(filename, mode='w', newline='') as file:
    # Create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header row
    writer.writeheader()
    
    # Write each dictionary as a row in the CSV file
    for person in ready_for_csv:
        writer.writerow(person)

print(f'CSV file "{filename}" has been created successfully.')


CSV file "final_output.csv" has been created successfully.
